# ISCE Processing of SnowEx UAVSAR data
By: **Jack Tarricone** *(University of Nevada, Reno)*, **Sam Neitlich** *(Montana State/JPL)*, **Talib Oliver** *(JPL)* \
Guidance from: **Yunling Lou** *(JPL)*, **Karen An** *(JPL)*, **Zach Keskenin** *(Boise State)*, **Ross Palomaki** *(Montana State)*, **Naheem Adebisi** *(Boise State)*

Create new conda environment to install ISCE in:
```
conda config --add channels conda-forge
conda config --set channel_priority strict
conda create -n isce2_env python=3.9 isce2=2.5.2 ** add numpy, glob, matplotlib
conda activate isce2_env
```

In the terminal, navigate to the home directory and open your ```.zprofile``` or ```.bash_profile```. In this case I'm using the [Atom](https://atom.io/) text editor:
```
cd ~
atom .zprofile
```

Add these lines or that path to where you ```conda``` enviroments are stored. Note that miniconda3 and be replace with anaconda3:
```
export ISCE_HOME=~/miniconda3/envs/isce2_env/lib/python3.9/site-packages/isce
export ISCE_STACK=~/miniconda3/envs/isce2_env/share/isce2/stripmapStack
export PATH=$PATH:$ISCE_HOME/bin:$ISCE_HOME/applications:$ISCE_STACK
```

Add link for copying extra pythons scripts here

Back in the terminal:
```
source .zprofile
```

In [1]:
%matplotlib inline
import isce
import os
import glob
import numpy as np
import shutil
from osgeo import gdal
import matplotlib.pyplot as plt
#import rasterio as rio
#import uavsar_pytools

This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce


This tutorial is a recreation of the processing steps outlined [IsceUAVSARprocessing.ipynb](https://github.com/taliboliver/Isce_UAVSAR/blob/master/IsceUAVSARprocessing.ipynb), with addtional functionality added by the ```uavsar_pytools``` package. It is assumed that users have ISCE installed on thier local machines, which admittedly can be the most challenging step in this process.  For information on ISCE installation please see the [Isce_UAVSAR](https://github.com/taliboliver/Isce_UAVSAR) by **Talib Oliver** *(JPL)* or Eric fieldings (find link from Zach)

In this notebook, we will interfermoterically process SLC stack from the Jemez Mountains, NM using the [Interferometric synthetic aperture radar Scientific Computing Environment (ISCE2)](https://github.com/isce-framework/isce2). We will then phase unwrap using [SNAPHU](https://web.stanford.edu/group/radar/softwareandlinks/sw/snaphu/), and geolocate the SLC data using ```geolocate_uavsar()```.

Past tutorials have assumed that your Jupyter environment will natively run command line code. In this notebook will run ISCE using mostly Python functionality.

## Create folder to work out of
First we'll create a new folder in our home directory to download files and run ISCE from

In [2]:
# get home dir
home = '/Users/jtarrico/'
os.chdir(home)
os.getcwd()

'/Users/jtarrico'

In [3]:
# working folder name
isce_directory = 'sierra_isce'
    
# path to folder
working_dir = os.path.join(home,isce_directory)

In [4]:
# make new folder
# os.mkdir(working_dir)
print("Directory '%s' created" %isce_directory)

Directory 'sierra_isce' created


In [5]:
# nativate to new folder
os.chdir(working_dir)
os.getcwd()

'/Users/jtarrico/sierra_isce'

In [6]:
## create subdirectories

# download
dl = 'download'
download_path = os.path.join(working_dir,dl)
# os.mkdir(download_path)

# slc
SLC = 'SLC'
slc_path = os.path.join(working_dir,SLC)
# os.mkdir(slc_path)

# baselines
bl = 'baselines'
baselines_path = os.path.join(working_dir,bl)
# os.mkdir(baselines_path)

# referenceShelve
rs = 'referenceShelve'
referenceShelve_path = os.path.join(working_dir,rs)
# os.mkdir(referenceShelve_path)

# dem
dem = 'DEM'
dem_path = os.path.join(working_dir,dem)
# os.mkdir(dem_path)

# igrams
igrams = 'igrams'
igrams_path = os.path.join(working_dir,igrams)
# os.mkdir(igrams_path)

In [7]:
# check folders were created
for file in os.listdir(working_dir):
    d = os.path.join(working_dir, file)
    if os.path.isdir(d):
        print(d)

/Users/jtarrico/sierra_isce/download_orginal
/Users/jtarrico/sierra_isce/SLC
/Users/jtarrico/sierra_isce/referenceShelve
/Users/jtarrico/sierra_isce/igrams
/Users/jtarrico/sierra_isce/download
/Users/jtarrico/sierra_isce/DEM
/Users/jtarrico/sierra_isce/baselines


## Data Download

Data links can be found on the UAVSAR site [here](https://uavsar.jpl.nasa.gov/cgi-bin/product.pl?jobName=alamos_35915_01#data"). In this case we'll be using SLCs from Feb 12th and 26th, both the LLH and LKV files, 1x1 pixel spacing, and just the HH polarization. By selecting these options, the site generates the list of ```wget``` download commands below.

 <p align="center">
  <img width="600" src="imgs/uavsar_data_portal.png">
</p>     
<p style="text-align: center;">
    (Data download screen)
</p>

Here the links are formatted as a string and saved as an exeutable shell file (.sh). This can also be down outside of this notebook by copying the links directly from the UAVSAR data portal and saving into your local text editor.

In [8]:
# nativate to download folder
os.chdir(download_path)
os.getcwd()

'/Users/jtarrico/sierra_isce/download'

In [9]:
# create .sh file from links

links = ['wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_01_BC.dop',
'wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_01_BC_s1_1x1.llh',
'wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_01_BC_s1_1x1.lkv',
'wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_01_BC_s2_1x1.llh',
'wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_01_BC_s2_1x1.lkv',
'wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_20002_001_200131_L090VV_01_BC.ann',
'wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_20006_002_200212_L090VV_01_BC.ann',
'wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_20009_023_200219_L090VV_01_BC.ann',
'wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_20016_005_200311_L090VV_01_BC.ann',
'wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_20014_000_200226_L090VV_01_BC.ann',
'wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_20002_001_200131_L090VV_01_BC_s1_1x1.slc',
'wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_20006_002_200212_L090VV_01_BC_s1_1x1.slc',
'wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_20009_023_200219_L090VV_01_BC_s1_1x1.slc',
'wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_20014_000_200226_L090VV_01_BC_s1_1x1.slc',
'wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_20016_005_200311_L090VV_01_BC_s1_1x1.slc',
'wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_20002_001_200131_L090VV_01_BC_s2_1x1.slc',
'wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_20006_002_200212_L090VV_01_BC_s2_1x1.slc',
'wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_20009_023_200219_L090VV_01_BC_s2_1x1.slc',
'wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_20014_000_200226_L090VV_01_BC_s2_1x1.slc',
'wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_20016_005_200311_L090VV_01_BC_s2_1x1.slc']

with open('downloads.sh', 'w') as f:
    for links in links:
        f.write(links)
        f.write('\n')

In [10]:
# ! sh downloads.sh

In [11]:
# list files downloaded
for i in os.listdir(download_path):
    file = os.path.join(download_path, i)
    print(file)

/Users/jtarrico/sierra_isce/download/sierra_17305_20009_023_200219_L090VV_01_BC.ann
/Users/jtarrico/sierra_isce/download/sierra_17305_01_BC_s2_1x1.lkv
/Users/jtarrico/sierra_isce/download/.DS_Store
/Users/jtarrico/sierra_isce/download/sierra_17305_01_BC.dop
/Users/jtarrico/sierra_isce/download/sierra_17305_20006_002_200212_L090VV_01_BC.ann
/Users/jtarrico/sierra_isce/download/sierra_17305_01_BC_s1_1x1.lkv
/Users/jtarrico/sierra_isce/download/sierra_17305_20014_000_200226_L090VV_01_BC.ann
/Users/jtarrico/sierra_isce/download/sierra_17305_01_BC_s1_1x1.llh
/Users/jtarrico/sierra_isce/download/sierra_17305_20016_005_200311_L090VV_01_BC.ann
/Users/jtarrico/sierra_isce/download/sierra_17305_20002_001_200131_L090VV_01_BC.ann
/Users/jtarrico/sierra_isce/download/sierra_17305_01_BC_s2_1x1.llh
/Users/jtarrico/sierra_isce/download/downloads.sh


## Prepare SLC Stack

In [13]:
# nativate to download folder
os.chdir(working_dir)
os.getcwd()

'/Users/jtarrico/sierra_isce'

In [14]:
# copy doppler file to working directory
shutil.copy("/Users/jtarrico/sierra_isce/download/sierra_17305_01_BC.dop", 
            "/Users/jtarrico/sierra_isce/sierra_17305_01_BC.dop")

'/Users/jtarrico/sierra_isce/sierra_17305_01_BC.dop'

In [ ]:
# check function inputs
! prepareUAVSAR_coregStack_segmets.py -h

In this case our -i flag will the the download folder, the -d flag will be the doppler file, -o will be the SLC directory.

In [ ]:
! prepareUAVSAR_coregStack_segmets.py --input download/ --dop_file sierra_17305_01_BC.dop --output SLC --segment 1

In [ ]:
# segment 2
! prepareUAVSAR_coregStack_segmets.py --input download/ --dop_file sierra_17305_01_BC.dop --output SLC --segment 2

In [15]:
# list folders just created in slc path
!ls SLC/*

SLC/20200131T1903:
data.db
sierra_17305_20002_001_200131_L090VV_01_BC.ann
sierra_17305_20002_001_200131_L090VV_01_BC_s1_1x1.slc
sierra_17305_20002_001_200131_L090VV_01_BC_s1_1x1.slc.vrt
sierra_17305_20002_001_200131_L090VV_01_BC_s1_1x1.slc.xml
sierra_17305_20002_001_200131_L090VV_01_BC_s2_1x1.slc
sierra_17305_20002_001_200131_L090VV_01_BC_s2_1x1.slc.vrt
sierra_17305_20002_001_200131_L090VV_01_BC_s2_1x1.slc.xml

SLC/20200212T2211:
data.db
sierra_17305_20006_002_200212_L090VV_01_BC.ann
sierra_17305_20006_002_200212_L090VV_01_BC_s1_1x1.slc
sierra_17305_20006_002_200212_L090VV_01_BC_s1_1x1.slc.vrt
sierra_17305_20006_002_200212_L090VV_01_BC_s1_1x1.slc.xml
sierra_17305_20006_002_200212_L090VV_01_BC_s2_1x1.slc
sierra_17305_20006_002_200212_L090VV_01_BC_s2_1x1.slc.vrt
sierra_17305_20006_002_200212_L090VV_01_BC_s2_1x1.slc.xml

SLC/20200219T2208:
data.db
sierra_17305_20009_023_200219_L090VV_01_BC.ann
sierra_17305_20009_023_200219_L090VV_01_BC_s1_1x1.slc
sierra_17305_20009_023_200219_L090VV_01_BC

In [16]:
os.getcwd()

'/Users/jtarrico/sierra_isce'

In [21]:
# using gdal info the check everything worked
! gdalinfo SLC/20200226T2253/sierra_17305_20014_000_200226_L090VV_01_BC_s2_1x1.slc

Driver: ISCE/ISCE raster
Files: SLC/20200226T2253/sierra_17305_20014_000_200226_L090VV_01_BC_s2_1x1.slc
       SLC/20200226T2253/sierra_17305_20014_000_200226_L090VV_01_BC_s2_1x1.slc.xml
Size is 8874, 63032
Coordinate System is:
GEOGCRS["WGS 84",
    DATUM["World Geodetic System 1984",
        ELLIPSOID["WGS 84",6378137,298.257223563,
            LENGTHUNIT["metre",1]]],
    PRIMEM["Greenwich",0,
        ANGLEUNIT["degree",0.0174532925199433]],
    CS[ellipsoidal,2],
        AXIS["geodetic latitude (Lat)",north,
            ORDER[1],
            ANGLEUNIT["degree",0.0174532925199433]],
        AXIS["geodetic longitude (Lon)",east,
            ORDER[2],
            ANGLEUNIT["degree",0.0174532925199433]],
    ID["EPSG",4326]]
Data axis to CRS axis mapping: 2,1
Origin = (0.000000000000000,0.000000000000000)
Pixel Size = (1.000000000000000,1.000000000000000)
Corner Coordinates:
Upper Left  (   0.0000000,   0.0000000) (  0d 0' 0.01"E,  0d 0' 0.01"N)
Lower Left  (       0.000,   63032.000) 

In [ ]:
ds = gdal.Open("/Users/jtarrico/sierra_isce/SLC/20200226T2253/sierra_17305_20014_000_200226_L090VV_01_BC_s2_1x1.slc", gdal.GA_ReadOnly)
# extract a part of the SLC to display
x0 = 0
y0 = 0
x_offset = 8874
y_offset = 63032 
slc = ds.GetRasterBand(1).ReadAsArray(x0, y0, x_offset, y_offset)
ds = None

fig = plt.figure(figsize=(14, 12))

# display amplitude of the slc
ax = fig.add_subplot(1,2,1)
ax.imshow(np.abs(slc), vmin = 0, vmax=1, cmap='gray')
ax.set_title("amplitude")

#display phase of the slc
ax = fig.add_subplot(1,2,2)
ax.imshow(np.angle(slc))
ax.set_title("phase")

plt.show()

slc = None

## Create Interferograms
more info here.....

In [ ]:
! uavsar_crossmul.py -h 

In [ ]:
! uavsar_crossmul.py --slc_dir SLC/ --alks 1 --rlks 1 --num_connections 1 --outdir igrams 

In [ ]:
# print current directories
! ls igrams/*/*

In [ ]:
# plot amplitude and wrapped interferogram

w_int = 'igrams/20200226T2253_20200311T1852/20200226T2253_20200311T1852.int'
ds = gdal.Open(w_int, gdal.GA_ReadOnly)
igram = ds.GetRasterBand(1).ReadAsArray()
ds = None

fig = plt.figure(figsize=(10, 14))

ax = fig.add_subplot(1,2,1)
ax.imshow(np.abs(igram), vmin = 0 , vmax = 8.0, cmap = 'gray')
ax.set_title("Magnitude")
ax.set_axis_off()

ax = fig.add_subplot(1,2,2)
ax.imshow(np.angle(igram), cmap = 'jet', vmax = 3.14, vmin = -3.14)
ax.set_title("Phase")
ax.set_axis_off()

# Phase Unwrapping

In [ ]:
# create text file of date list
## this doesn't work, had to fix manually
! ls igrams/ | awk -F "/" '{print $1}' >> datelist.txt

In [ ]:
# print contents of that file
with open('datelist.txt') as f:
    lines = f.readlines()
    print(lines)

In [ ]:
! unwrap.py -h

In [ ]:
# run snaphu, this will take a while
! awk -F "_" '{print "unwrap.py -i igrams/"$1"_"$2"/"$1"_"$2".int -u igrams/"$1"_"$2"/"$1"_"$2".unw -c igrams/"$1"_"$2"/"$1"_"$2".coh -a 12 -r 3 -s SLC/"$1"/ -m snaphu"}' datelist.txt >> unwall.sh

In [ ]:
! sh unwall.sh

In [ ]:
# list files generated in unwrapping process
!ls igrams/*/*

In [ ]:
# plot

# reading the multi-looked wrapped interferogram
ds = gdal.Open("igrams/20200226T2253_20200311T1852/20200226T2253_20200311T1852.int", gdal.GA_ReadOnly)
igram = ds.GetRasterBand(1).ReadAsArray()
ds = None

# reading the multi-looked unwrapped interferogram
ds = gdal.Open("igrams/20200226T2253_20200311T1852/20200226T2253_20200311T1852.unw_snaphu.unw", gdal.GA_ReadOnly)
igram_unw = ds.GetRasterBand(2).ReadAsArray()
ds = None

# reading the connected component file
ds = gdal.Open("igrams/20200226T2253_20200311T1852/20200226T2253_20200311T1852.unw_snaphu.unw.conncomp", gdal.GA_ReadOnly)
connected_components = ds.GetRasterBand(1).ReadAsArray()
ds = None

fig = plt.figure(figsize=(18, 16))

ax = fig.add_subplot(1,3,1)
cax=ax.imshow(np.angle(igram), cmap='jet')
ax.set_title("wrapped")
#ax.set_axis_off()
cbar = fig.colorbar(cax, ticks=[-3.14,0,3.14],orientation='horizontal')
cbar.ax.set_xticklabels(["$-\pi$",0,"$\pi$"])

ax = fig.add_subplot(1,3,2)
cax = ax.imshow(igram_unw, vmin = 0, vmax = 7, cmap = 'jet')
ax.set_title("unwrapped")
ax.set_axis_off()
cbar = fig.colorbar(cax, ticks=[0,3.5,7], orientation='horizontal')


ax = fig.add_subplot(1,3,3)
cax = ax.imshow(connected_components, cmap = 'jet')
ax.set_title("components")
ax.set_axis_off()
cbar = fig.colorbar(cax, ticks=[0,21] , orientation='horizontal')
cbar.ax.set_xticklabels([0,21])


connected_components = None